In [54]:
import json

class DrukharyGraph:
    nodesIndex:list
    nodesNames:dict
    edges:list[list]

    def __init__(self,path,nodeNames,edgesName,nodeFrom,nodeTo,connectName,isBidirectional) -> None:

        with open(path) as f:
            data = json.load(f)

            self.nodesIndex = list(set(data.get(nodeNames)))
            self.nodesNames = dict([[self.nodesIndex[i],i] for i in range(len(self.nodesIndex))])
            
            self.edges = list([[None for j in range(len(self.nodesIndex))] for i in range(len(self.nodesIndex))])
            for edge in data.get(edgesName):
                self.edges[self.nodesNames[edge[nodeFrom]]][self.nodesNames[edge[nodeTo]]] = edge[connectName]
                if isBidirectional:
                    self.edges[self.nodesNames[edge[nodeTo]]][self.nodesNames[edge[nodeFrom]]] = edge[connectName]

    def mapNodesToNames(self,path):
        return [self.nodes[i] for i in path]


    # getters
    def getNodes(self):
        return [i for i in range(len(self.nodesNames))]

    def getEdges(self):
        return [i.copy() for i in self.edges]


    def getNodeByIndex(self,index:int):
        return self.nodesIndex[index]

    def getNodeByName(self,name:str):
        return self.nodesNames[name]


    def mapIndexToNames(self,nodes):
        return [self.getNodeByIndex(node) for node in nodes]


    def searchBy(self,method,source,destination,**kwargs):
        return method(self.edges,self.getNodeByName(source),self.getNodeByName(destination),**kwargs)


    

cities = DrukharyGraph("cities.json","cities","connections","city1","city2","distance",True)


Воронеж
24


In [ ]:
from cmath import inf


class NonInformedSearching:


    @staticmethod
    def BFS(connections:list,source:int,destination:int,**kwargs):
        if source == destination:
            return 0



        visited = [False for i in range(len(connections))]
        visited[source] = True
        nodes = [source]
        current_node = 0
        isFound = False
        while len(nodes) >= current_node and not isFound:

            for j in range(len(connections)):
                if connections[nodes[current_node]][j] is not None and not visited[j]:
                    nodes.append(j)
                    visited[j] = True
                    if (destination==j):
                        isFound = True
                        break

            current_node += 1


        path = [destination]
        for i in nodes.__reversed__():
            if connections[i][path[-1]] is not None:
                path.append(i)

        return path.__reversed__()


    @staticmethod
    def DFS(connections:list,source:int,destination:int,**kwargs):
        if source == destination:
            return [source]

        visited = [False for i in connections]
        path = [source]
        current_node = 0
        while len(path) > 0:
            j = 0
            while j < len(connections) and (connections[path[current_node]][j] is None or visited[j]):
                j += 1
            if j < len(connections):
                path.append(j)
                visited[j] = True
                current_node += 1
            else:
                path.pop()
                current_node -= 1

            if (path[current_node] == destination):
                return [i for i in path]
            
        return []
    
    @staticmethod
    def DFSL(connections:list,source:int,destination:int,**kwargs):

        if source == destination:
            return [source]

        limit = kwargs["limit"] if "limit" in kwargs else 10


        visited = [ [False if j else None for j in i] for i in connections]
        visitedPath = [False for i in connections]
        path = [source]
        visitedPath[source] = True
        current_node = 0
        while len(path) > 0:

            j = 0
            while j<len(connections) and ( connections[path[current_node]][j] is None or visited[path[current_node]][j] or  visitedPath[j]):
                j += 1

            if j < len(connections) and len(path) < limit:
                # print(j)

                path.append(j)
                visited[path[current_node]][j] = True
                visitedPath[j] = True
                current_node += 1
            else:
                visitedPath[path.pop()] = False
                current_node -= 1

            if len(path) > 0 and path[current_node] == destination:
                return [i for i in path]

            
        return []


    @staticmethod
    def DFSIL(connections:list,source:int,destination:int,**kwargs):
        max = kwargs["max"] if "max" in kwargs else 5

        for i in range(2,max):
            path = NonInformedSearching.DFSL(connections,source,destination,limit=i)
            if (len(path)>0):
                return path
        return []

    @staticmethod
    def BS(connections:list,source:int,destination:int,**kwargs):
        if source == destination:
            return [source]



        visitedS = [False for i in range(len(connections))]
        visitedS[source] = True
        nodesS = [source]
        currentNodeS = 0

        visitedD = [False for i in range(len(connections))]
        visitedD[destination] = True
        nodesD = [destination]
        currentNodeD = 0

        isFound = False

        meetingPlace = None
        while len(nodesS) >= currentNodeS and len(nodesD) >= currentNodeD and not isFound:

            for j in range(len(connections)):
                if connections[nodesS[currentNodeS]][j] is not None and not visitedS[j]:
                    nodesS.append(j)
                    visitedS[j] = True
                    if visitedD[j]:
                        meetingPlace = j
                        isFound = True
                        break
                if connections[nodesD[currentNodeD]][j] is not None and not visitedD[j]:
                    nodesD.append(j)
                    visitedD[j] = True
                    if visitedS[j]:
                        meetingPlace = j
                        isFound = True
                        break
            
            currentNodeS += 1
            currentNodeD += 1

        if meetingPlace is None:
            return []

        pathS = [nodesS[-1]]
        pathD = [nodesD[-1]]

        for i in nodesS.__reversed__():
            if connections[i][pathS[-1]] is not None:
                pathS.append(i)
        for i in nodesD.__reversed__():
            if connections[i][pathD[-1]] is not None:
                pathD.append(i)
        
        pathS = list(pathS.__reversed__())
        pathS.pop()
        
        return  pathS + pathD



class GraphAlgs:
    @staticmethod
    def dijkstra(connections,destination):
        visited = [False for i in connections]
        nodes = [destination]

        current_node_index = 0
        distances = [inf for i in connections]
        distances[destination] = 0  
        
        while current_node_index < len(nodes):
            current_node = nodes[current_node_index]

            for i in range(len(connections)):
                if connections[current_node][i] is not None and not visited[i]:
                    if distances[current_node] + connections[current_node][i]  < distances[i]:
                        distances[i] = distances[current_node] + connections[current_node][i]

            visited[current_node] = True
            current_node_index += 1
            i = 0
            for i in range(len(visited)):
                if not visited[i] and distances[i] is not inf:
                    nodes.append(i)
                    break

        return distances

class InformedSearching:
    @staticmethod
    def greedy(connections,source,destination,**kwargs):

        if source == destination:
            return [source]

        distances = GraphAlgs.dijkstra(connections,destination)

        visited = [False for i in connections]
        visited[source] = True
        path = [source]
        currentNodeIndex = 0
        while len(path) > 0:
            currentNode = path[currentNodeIndex]

            currentNode = path[currentNodeIndex]
            nextNodeIndex = None
            for j in range(len(connections)):
                if connections[currentNode][j] is not None and not visited[j]:
                    if nextNodeIndex is None:
                        nextNodeIndex = j
                    else:
                        if distances[nextNodeIndex] > distances[j]:
                            nextNodeIndex = j

            if nextNodeIndex is not None and nextNodeIndex < len(connections):
                path.append(nextNodeIndex)
                visited[nextNodeIndex] = True
                currentNodeIndex += 1
            else:
                path.pop()
                currentNodeIndex -= 1

            if path[currentNodeIndex] == destination:
                return [i for i in path]

            
            
        return path
    def greedyAStar(connections,source,destination,**kwargs):

        if source == destination:
            return [source]

        distancesD = GraphAlgs.dijkstra(connections,destination)
        distancesS = GraphAlgs.dijkstra(connections, source)
        visited = [False for i in connections]
        visited[source] = True
        path = [source]
        currentNodeIndex = 0
        while len(path) > 0:
            currentNode = path[currentNodeIndex]

            currentNode = path[currentNodeIndex]
            nextNodeIndex = None
            for j in range(len(connections)):
                if connections[currentNode][j] is not None and not visited[j]:
                    if nextNodeIndex is None:
                        nextNodeIndex = j
                    else:
                        if distancesD[nextNodeIndex] + distancesS[nextNodeIndex] > distancesD[j] + distancesS[j]:
                            nextNodeIndex = j

            if nextNodeIndex is not None and nextNodeIndex < len(connections):
                path.append(nextNodeIndex)
                visited[nextNodeIndex] = True
                currentNodeIndex += 1
            else:
                path.pop()
                currentNodeIndex -= 1

            if path[currentNodeIndex] == destination:
                return [i for i in path]

            
            
        return path

In [68]:


def testSearching(source,destination):
    for method,descrb in [[NonInformedSearching.BFS,"Breadth-first search"],
                            [NonInformedSearching.DFS, "Depth-first search"],
                            [NonInformedSearching.DFSL, "Breadth-first search with limitation of depth"],
                            [NonInformedSearching.DFSIL, "Breadth-first search with iteration deepening"],
                            [NonInformedSearching.BS,"Bidirectional search"]
                            ]:
        path = cities.searchBy(method,source=source,destination=destination,max=10)
        print(f"the {descrb} is {cities.mapIndexToNames(path)}")

    for method,descrb in [[InformedSearching.greedy,"Greedy search by first best accordance"],
                            [InformedSearching.greedyAStar, "Search A*"],
                            ]:
        path = cities.searchBy(method,source=source,destination=destination,max=10)
        print(f"The result of {descrb} is {cities.mapIndexToNames(path)}")






# testSearching("Самара","Казань")

# testSearching("Витебск","Казань")

testSearching("Симферополь","Таллинн")
# testSearching("Витебск","Рига")

# 
# testSearching("Витебск","Ярославль")



the find in  is ['Симферополь', 'Харьков', 'Киев', 'Житомир', 'Волгоград', 'Витебск', 'Орел', 'Москва', 'Санкт-Петербург', 'Рига', 'Таллинн']
the find of is ['Симферополь', 'Харьков', 'Киев', 'Кишинев', 'Донецк', 'Орел', 'Витебск', 'Воронеж', 'Ярославль', 'Минск', 'Мурманск', 'Санкт-Петербург', 'Калининград', 'Вильнюс', 'Каунас', 'Рига', 'Таллинн']
the just find is ['Симферополь', 'Харьков', 'Киев', 'Кишинев', 'Донецк', 'Орел', 'Витебск', 'Санкт-Петербург', 'Рига', 'Таллинн']
the finfidul is ['Симферополь', 'Харьков', 'Киев', 'Вильнюс', 'Каунас', 'Рига', 'Таллинн']
the asdas is ['Симферополь', 'Харьков', 'Киев', 'Вильнюс', 'Каунас', 'Рига', 'Таллинн']
the greedy  is ['Симферополь', 'Харьков', 'Киев', 'Вильнюс', 'Каунас', 'Рига', 'Таллинн']
the greedy A* is ['Симферополь', 'Харьков', 'Киев', 'Вильнюс', 'Каунас', 'Рига', 'Таллинн']
